In [ ]:
# 10_tageament_avancado_colab.ipynb

# Instalação de bibliotecas (no ambiente do Colab)
!pip install pandas transformers matplotlib tqdm

# Imports
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

# Ajustes de visualização
tqdm.pandas()
plt.rcParams['figure.figsize'] = (10, 6)

# Verifica os arquivos no diretório atual para garantir que o CSV está no local correto
print("Arquivos no diretório atual:", os.listdir('/content'))

# 1. Leitura do arquivo CSV
caminho_csv = '/content/dados_aosfatos_tematico_enriquecido.csv'
df = pd.read_csv(caminho_csv, parse_dates=['data_publicacao'])

# 2. Preparar o modelo zero-shot (download automático no Colab)
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

# 3. Definir os temas (labels)
labels = [
    'política', 'governo', 'presidente', 'economia', 'saúde', 'covid', 'pandemia',
    'fake news', 'desinformação', 'redes sociais', 'facebook', 'inteligência artificial',
    'fraudes digitais', 'autoritarismo', 'discurso de ódio', 'negacionismo climático',
    'exploração de tragédias', 'revisionismo histórico', 'políticas públicas', 'tecnopolítica'
]

# 4. Função de classificação
def classificar_tema(texto):
    try:
        resultado = classifier(texto, candidate_labels=labels)
        return resultado['labels'][0]
    except:
        return 'erro'

# 5. Aplicar a classificação (para o primeiro teste, limitamos a 100 registros)
df_sample = df.head(100).copy()
df_sample['tema_predito'] = df_sample['conteudo_completo'].fillna('').progress_apply(classificar_tema)

# 6. Salvar o resultado de teste
df_sample.to_csv('/content/resultado_teste_classificacao.csv', index=False)
print("Arquivo resultado_teste_classificacao.csv salvo com sucesso.")

# 7. Gráfico de quantitativo de temas

# Contagem de temas preditos
tema_counts = df_sample['tema_predito'].value_counts()

plt.bar(tema_counts.index, tema_counts.values)
plt.xticks(rotation=90)
plt.title('Distribuição dos temas (teste)')
plt.ylabel('Quantidade')
plt.xlabel('Tema')
plt.show()

# 8. Gráfico de variação ao longo do ano

df_sample['ano'] = df_sample['data_publicacao'].dt.year

tema_ano = df_sample.groupby(['ano', 'tema_predito']).size().unstack(fill_value=0)

tema_ano.plot(kind='bar', stacked=True)
plt.title('Variação dos temas por ano (teste)')
plt.ylabel('Quantidade')
plt.xlabel('Ano')
plt.show()
